In [3]:
# Imports and helper setup
import os
import sys
from passlib.context import CryptContext

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Import app DB/session and models
from app.database import SessionLocal
from app import models

pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")


def create_user(email: str, password: str, username: str | None = None):
    """Create a user in the DB and return the created user object."""
    db = SessionLocal()
    try:
        hashed = pwd_context.hash(password)
        user = models.User(email=email, username=username or email.split("@")[0], hashed_password=hashed)
        db.add(user)
        db.commit()
        db.refresh(user)
        return {"id": user.id, "email": user.email, "username": user.username}
    finally:
        db.close()

### Notes

- Run this notebook from `backend/` or set `PYTHONPATH=.` so `import app` resolves.
- Ensure `DATABASE_URL` is set in your environment and migrations applied.
- This is intended for development only; don't store secrets in versioned notebooks.


In [4]:
# Parameters - change these values before running
email = "user@example.com"
password = "P4ssw0rd"
username = "bart"

# Create the user and show result
result = create_user(email, password, username)
result

(trapped) error reading bcrypt version
Traceback (most recent call last):
  File "/home/rozekb/Documents/code/cookapp/cookapp/backend/.venv/lib/python3.11/site-packages/passlib/handlers/bcrypt.py", line 620, in _load_backend_mixin
    version = _bcrypt.__about__.__version__
              ^^^^^^^^^^^^^^^^^
AttributeError: module 'bcrypt' has no attribute '__about__'


IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "users_username_key"
DETAIL:  Key (username)=(bart) already exists.

[SQL: INSERT INTO users (username, email, hashed_password, is_active, is_superuser, is_verified, created_at) VALUES (%(username)s, %(email)s, %(hashed_password)s, %(is_active)s, %(is_superuser)s, %(is_verified)s, %(created_at)s) RETURNING users.id]
[parameters: {'username': 'bart', 'email': 'user@example.com', 'hashed_password': '$2b$12$ebZ7x4wPbnbi./Oovbx0ruk4dSLjmmXJzuqnqSMMzDWceOVhGLvQu', 'is_active': True, 'is_superuser': False, 'is_verified': False, 'created_at': datetime.datetime(2025, 8, 18, 17, 31, 8, 95842, tzinfo=datetime.timezone.utc)}]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

## Notebook settings

Target kernel: python3

Libraries used: passlib, sqlalchemy, pydantic (optional). If a library is missing, install it in your environment (avoid installing packages in this notebook unless you are in an isolated environment).

# Create user helper

This notebook converts `backend/scripts/create_user.py` into an interactive notebook. It creates a single user in the database using the app's SQLAlchemy models and the project's `SessionLocal`.

Prerequisites:
- Set `DATABASE_URL` in your environment (see `backend/.env` or your environment variables).
- Run Alembic migrations so the `users` table exists.
- Run this notebook from the `backend/` directory or set `PYTHONPATH=.` so `app` package imports resolve.
